
# 🧠 Dense vs Active Parameter (Local) — Phi-3 Mini (Dense) vs Mixtral (GPTQ MoE)

This lab compares a **Dense Model** (`microsoft/phi-3-mini-4k-instruct`) and an **Active-Parameter / MoE Model** (`TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ`), both loaded **locally** through Hugging Face Transformers.

---

### 🎯 Objectives
- Understand dense vs active parameter model architecture
- Run reasoning, summarization, knowledge recall, creativity, and code benchmarks
- Observe qualitative differences in model performance and response structure

### ⚙️ Requirements
- Python ≥ 3.10
- `transformers`, `accelerate`, `bitsandbytes`, and optionally `auto-gptq`
- GPU recommended (16GB+ for Mixtral GPTQ), CPU or MPS supported for Phi-3

---


In [ ]:

# Install dependencies (uncomment if needed)
# !pip install -q transformers accelerate bitsandbytes auto-gptq

import torch, time, platform
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Python:", platform.python_version())
print("PyTorch:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else ("mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "cpu"))
print("✅ Using device:", device)


In [ ]:

# Define model IDs
PHI_MODEL = "microsoft/phi-3-mini-4k-instruct"
MIXTRAL_MODEL = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ"

print("Dense model:", PHI_MODEL)
print("Active parameter (MoE) model:", MIXTRAL_MODEL)


In [ ]:

from transformers import GenerationConfig

def set_pad_token_if_missing(tokenizer):
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

def make_generator(model, tokenizer, device):
    def generate(prompt, max_new_tokens=200, temperature=0.7, top_p=0.9):
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        start = time.time()
        with torch.no_grad():
            output_tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, pad_token_id=tokenizer.eos_token_id)
        elapsed = time.time() - start
        text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        return text, elapsed
    return generate



## 🧪 Benchmarks

We will evaluate both models across these tasks:
1. **Reasoning**
2. **Summarization**
3. **Knowledge Recall**
4. **Creative Writing**
5. **Code Generation**


In [ ]:

benchmarks = {
    "Reasoning": "A train travels 60 km at 60 km/h, then another 60 km at 120 km/h. What is the average speed for the whole trip?",
    "Summarization": "Summarize this paragraph in two lines:\n\nHPE ProLiant servers deliver secure, software-defined compute with HPE OneView and HPE InfoSight.",
    "Knowledge Recall": "Who proposed the theory of general relativity and when?",
    "Creative Writing": "Write a short 4-line poem about artificial intelligence helping humanity.",
    "Code Generation": "Write a Python function `factorial(n)` that returns n! using recursion, with a docstring."
}
print("Loaded benchmarks:", list(benchmarks.keys()))



## 🧩 Phi-3 Mini (Dense Model)

Phi-3 Mini is a **dense transformer** — all its parameters are active during every forward pass.  
It’s efficient for small systems and ideal as a lightweight dense baseline.


In [ ]:

phi_tokenizer = AutoTokenizer.from_pretrained(PHI_MODEL)
set_pad_token_if_missing(phi_tokenizer)

phi_model = AutoModelForCausalLM.from_pretrained(PHI_MODEL, torch_dtype=torch.float16 if device.type=="cuda" else torch.float32, device_map="auto")
phi_gen = make_generator(phi_model, phi_tokenizer, device)

phi_outputs = {}

for task, prompt in benchmarks.items():
    print(f"\n### Task: {task}")
    output, t = phi_gen(prompt, max_new_tokens=200)
    phi_outputs[task] = output
    print(f"🕒 Time: {t:.2f}s\n{output}\n")



## ⚙️ Mixtral (Active Parameter / MoE Model)

Mixtral is a **Mixture of Experts (MoE)** model — it contains multiple subnetworks ("experts") and activates only a subset per token.  
This makes it **computationally efficient** while retaining high capacity.

> Note: Quantized Mixtral (GPTQ) version reduces memory usage and allows inference on smaller GPUs.


In [ ]:

mix_tokenizer = AutoTokenizer.from_pretrained(MIXTRAL_MODEL, trust_remote_code=True)
set_pad_token_if_missing(mix_tokenizer)

mix_model = AutoModelForCausalLM.from_pretrained(MIXTRAL_MODEL, device_map="auto", trust_remote_code=True)
mix_gen = make_generator(mix_model, mix_tokenizer, device)

mix_outputs = {}

for task, prompt in benchmarks.items():
    print(f"\n### Task: {task}")
    output, t = mix_gen(prompt, max_new_tokens=200)
    mix_outputs[task] = output
    print(f"🕒 Time: {t:.2f}s\n{output}\n")



## 🧾 Observations and Notes

- **Dense Model (Phi-3 Mini)**: Uses all weights every step. Smaller, consistent, but limited capacity.  
- **Active Model (Mixtral)**: Uses expert routing, activating only a subset of parameters per token, achieving better efficiency and often higher quality.

### Next steps
- Add metrics like BLEU/ROUGE or factual accuracy comparisons.  
- Try other models (e.g., `Llama-3-8B` vs. `Mixtral-8x7B`) for larger-scale testing.
